<a href="https://colab.research.google.com/github/ciccioshake/colab/blob/main/testo_segmentato_ottimizzato_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install chardet
import re
import chardet
import nltk
from nltk.tokenize import word_tokenize

# Scarica il tokenizer di NLTK se necessario
nltk.download("punkt")

# Impostazioni per la segmentazione migliorata
MAX_LEN = 384  # Lunghezza massima di un segmento
OVERLAP = 100  # Sovrapposizione tra segmenti
PUNCTUATION = [".", "!", "?", ";", ":", ","]  # Punti di spezzatura preferiti
CHUNK_SIZE = 1024 * 1024  # Dimensione del blocco di lettura (1 MB)


# Funzione per tokenizzare il testo in parole senza dipendenze esterne
def simple_tokenize(text):
    return re.findall(r'\b\w+\b', text)


# Funzione per trovare il miglior punto di spezzatura
def find_best_split(words, max_len):
    """
    Cerca il miglior punto di spezzatura in base alla punteggiatura.
    """
    if len(words) <= max_len:
        return len(words)  # Nessuna spezzatura necessaria

    best_split = max_len  # Punto di spezzatura predefinito
    for i in range(max_len - 10, max_len - 50, -1):  # Cerca un punto vicino alla fine
        if words[i] in PUNCTUATION:
            best_split = i + 1  # Include la punteggiatura nel segmento
            break
    return best_split


# Percorsi file aggiornati
input_file_path = "/content/testo_input.txt"
output_file_path = "/content/testo_segmentato_ottimizzato.txt"


# Elaborazione del file in blocchi
with open(input_file_path, "r", encoding="ISO-8859-1") as f, open(output_file_path, "w", encoding="utf-8") as outfile:
    buffer = []  # Buffer per accumulare parole
    segment_count = 0

    while True:
        chunk = f.read(CHUNK_SIZE)
        if not chunk:
            break  # Fine del file

        # Tokenizza il blocco corrente
        words = simple_tokenize(chunk)
        buffer.extend(words)

        # Segmentazione del buffer
        while len(buffer) >= MAX_LEN:
            end = find_best_split(buffer, MAX_LEN)
            segment = buffer[:end]
            segment_count += 1
            outfile.write(
                f"Segment {segment_count} (Token Count: {len(segment)}):\n{' '.join(segment)}\n\n"
            )
            buffer = buffer[end - OVERLAP :]  # Sovrapposizione

    # Elabora eventuali parole rimanenti nel buffer
    if buffer:
        segment_count += 1
        outfile.write(
            f"Segment {segment_count} (Token Count: {len(buffer)}):\n{' '.join(buffer)}\n\n"
        )

print(f"Testo elaborato salvato in {output_file_path}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Testo elaborato salvato in /content/testo_segmentato_ottimizzato.txt
